In [ ]:


import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define the stock symbol and download historical data
stock_symbol = 'AAPL'

data = yf.download(stock_symbol, period="max)

# Extract the 'Close' prices
close_prices = data['Close'].values.reshape(-1, 1)

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
close_prices_scaled = scaler.fit_transform(close_prices)

# Function to create input data for LSTM
def create_lstm_data(data, time_steps=1):
    x, y = [], []
    for i in range(len(data) - time_steps):
        x.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(x), np.array(y)

# Set the time steps for the LSTM model
time_steps = 10

# Create input data for LSTM
x, y = create_lstm_data(close_prices_scaled, time_steps)

# Reshape the data for LSTM input [samples, time steps, features]
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x, y, epochs=50, batch_size=32)

# Predict future stock prices
future_dates = pd.date_range(start=end_date, periods=30)
future_dates = future_dates[1:]  # Remove the first date (already in the dataset)

# Get the last 10 closing prices to predict the future prices
last_prices = close_prices[-time_steps:]
last_prices_scaled = scaler.transform(last_prices.reshape(-1, 1))

# Create input data for predicting the future prices
x_pred = np.array([last_prices_scaled[-time_steps:, 0]])
x_pred = np.reshape(x_pred, (x_pred.shape[0], x_pred.shape[1], 1))

# Predict the future prices
predicted_prices_scaled = model.predict(x_pred)
predicted_prices = scaler.inverse_transform(predicted_prices_scaled)

# Create a DataFrame with the future dates and predicted prices
future_data = pd.DataFrame({'Date': future_dates, 'Predicted Price': predicted_prices.flatten()})

print(future_data)
     
